# BLEU ROGUE on Code Summary vs BR summary for COBOL code

In [2]:
!pip install evaluate

In [3]:
import evaluate
import os


In [4]:
code_summary_directory = "../Test_COBOLs/Output_Code_Summary"
BR_summary_directory = "../Test_COBOLs/Output_BR_Summary"

## Reading file content

In [5]:
def read_files(directory):
    file_contents = {}
    for filename in os.listdir(directory):
        if filename.endswith('.md'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                lines = [line.rstrip('\n') for line in file.readlines() if line.strip()]
                file_contents[filename] = lines
    return file_contents

code_summary_list = read_files(code_summary_directory)
br_summary_list = read_files(BR_summary_directory)

In [6]:
code_summary_list['Out_Car_Rental.md']

['## COBOL Code Analysis: Rental Car Payment Calculator (PROD2V1) ',
 'The provided COBOL code, labeled PROD2V1, functions as a payment calculation program for a rental car service. It reads input data from a file, performs calculations based on the input, and generates an output file with the calculated payment details. Below is a detailed breakdown of its structure and functionality:',
 '**Data Division:**',
 '*   **File Section:**',
 '    *   **RENTAL:** Sequential input file containing rental-related information. Fields include client name, rental type (initial, car type, kilometers traveled, number of days rented), and more.',
 '    *   **RENTAL-OUT:** Sequential output file to store calculated payment and updated rental information. ',
 '*   **Working-Storage Section:**',
 '    *   Various numeric and character variables used for intermediate calculations and storage of results, such as `KILOMETERS_PAYMENT`, `RENTAL_DAYS_TOTAL`, and more.',
 '**Procedure Division:**',
 '1.  **Mai

In [7]:
br_summary_list["Out_Car_Rental.md"]

['# Business Rules Summary',
 'This document outlines four business rules related to rental car fee calculations.',
 '## Rule 1: Initial Kilometer Subtraction (BR-001)',
 '* Description: This rule subtracts the first 75 kilometers from the total kilometers, if applicable.',
 '* Condition: `KILOMETERS > 75`',
 '* Output:',
 '\t* If the condition is met: `KILOMETERS = KILOMETERS - 75`',
 '## Rule 2: Volkswagen Cars Payment Calculation (BR-002)',
 '* Description: This rule calculates the payment for Volkswagen cars.',
 '* Condition: `CAR_TYPE = 1`',
 '* Output:',
 '\t* Kilometer Payment: `KILOMETERS_PAYMENT = KILOMETERS * 0.5`',
 '\t* Rental Days Payment: `RENTAL_DAYS_TOTAL = 10 * NUM_DAYS`',
 '\t* Total Payment: `PAYMENT = KILOMETERS_PAYMENT + RENTAL_DAYS_TOTAL`',
 '## Rule 3: Toyota Cars Payment Calculation (BR-003)',
 '* Description: This rule calculates the payment for Toyota cars.',
 '* Condition: `CAR_TYPE = 2`',
 '* Output:',
 '\t* Kilometer Payment: `KILOMETERS_PAYMENT = KILOMETER

In [8]:
print(len(code_summary_list))
print(len(br_summary_list))

12
12


## Loading the BLEU module and evaluating

In [9]:
bleu = evaluate.load("bleu")
sum_bleu = 0
for file_name in code_summary_list:
    reference = code_summary_list[file_name]
    prediction = br_summary_list[file_name]
    min_size = min(len(reference), len(prediction))
    reference = reference[:min_size]
    prediction = prediction[:min_size]
    
    result = bleu.compute(predictions = prediction, references = reference)
    sum_bleu += result['bleu']
    print(f"File name:{file_name}\nResult:{result}\n\n")

avg_bleu = sum_bleu/len(code_summary_list)
print(f"Average BLEU score:{avg_bleu}")

File name:Out_Car_Rental.md
Result:{'bleu': 0.0, 'precisions': [0.15337423312883436, 0.006711409395973154, 0.0, 0.0], 'brevity_penalty': 0.6292729434506871, 'length_ratio': 0.6834381551362684, 'translation_length': 326, 'reference_length': 477}


File name:Out_ClassCondition.md
Result:{'bleu': 0.0, 'precisions': [0.3125, 0.04895104895104895, 0.015873015873015872, 0.0], 'brevity_penalty': 0.3587964654059516, 'length_ratio': 0.49382716049382713, 'translation_length': 160, 'reference_length': 324}


File name:Out_cobol-calc.md
Result:{'bleu': 0.0, 'precisions': [0.24324324324324326, 0.0, 0.0, 0.0], 'brevity_penalty': 0.42876476368382394, 'length_ratio': 0.5414634146341464, 'translation_length': 222, 'reference_length': 410}


File name:Out_ConditionNameCondition.md
Result:{'bleu': 0.09224478976144551, 'precisions': [0.3333333333333333, 0.13592233009708737, 0.10526315789473684, 0.06896551724137931], 'brevity_penalty': 0.6849712733063196, 'length_ratio': 0.7254901960784313, 'translation_len

## Loading the ROGUE module and evaluating

In [10]:
!pip install rouge_score

In [11]:
rouge = evaluate.load("rouge")
sum_rouge1, sum_rouge2, sum_rougeL, sum_rougeLsum = 0, 0, 0, 0
for file_name in code_summary_list:
    reference = code_summary_list[file_name]
    prediction = br_summary_list[file_name]
    min_size = min(len(reference), len(prediction))
    reference = reference[:min_size]
    prediction = prediction[:min_size]
    result = rouge.compute(predictions = prediction, references = reference)
    
    sum_rouge1 += result['rouge1']
    sum_rouge2 += result['rouge2']
    sum_rougeL += result['rougeL']
    sum_rougeLsum += result['rougeLsum']
    
    print(f"File name:{file_name}\nResult:{result}\n\n")
    
avg_rouge1 = sum_rouge1/len(code_summary_list)
avg_rouge2 = sum_rouge2/len(code_summary_list)
avg_rougeL = sum_rougeL/len(code_summary_list)
avg_rougeLsum = sum_rougeLsum/len(code_summary_list)

print(f"Average rouge1:{avg_rouge1}")
print(f"Average rouge2:{avg_rouge2}")
print(f"Average rouge1:{avg_rouge1}")
print(f"Average rouge1:{avg_rouge1}")


File name:Out_Car_Rental.md
Result:{'rouge1': 0.07515326030077582, 'rouge2': 0.0057823129251700685, 'rougeL': 0.07474626990725085, 'rougeLsum': 0.07447336794249472}


File name:Out_ClassCondition.md
Result:{'rouge1': 0.07701734711716562, 'rouge2': 0.017202174987642118, 'rougeL': 0.06823703407054532, 'rougeLsum': 0.06766781243401293}


File name:Out_cobol-calc.md
Result:{'rouge1': 0.04616330097273836, 'rouge2': 0.0, 'rougeL': 0.04260311846518743, 'rougeLsum': 0.042479435505103136}


File name:Out_ConditionNameCondition.md
Result:{'rouge1': 0.1344211344211344, 'rouge2': 0.061155913978494625, 'rougeL': 0.11231060606060606, 'rougeLsum': 0.11590423465423465}


File name:Out_CWBWDATE.md
Result:{'rouge1': 0.06460672475378358, 'rouge2': 0.024166666666666666, 'rougeL': 0.05356591121297004, 'rougeLsum': 0.05234324278441925}


File name:Out_FIZZBUZZ.md
Result:{'rouge1': 0.08077578869333628, 'rouge2': 0.006993006993006992, 'rougeL': 0.07298324531939755, 'rougeLsum': 0.0733226862402338}


File name